In [24]:
import pandas as pd
import numpy as np

df = pd.read_csv('data/WA_Fn-UseC_-Telco-Customer-Churn.csv')
print(df.head())


   customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  7590-VHVEG  Female              0     Yes         No       1           No   
1  5575-GNVDE    Male              0      No         No      34          Yes   
2  3668-QPYBK    Male              0      No         No       2          Yes   
3  7795-CFOCW    Male              0      No         No      45           No   
4  9237-HQITU  Female              0      No         No       2          Yes   

      MultipleLines InternetService OnlineSecurity  ... DeviceProtection  \
0  No phone service             DSL             No  ...               No   
1                No             DSL            Yes  ...              Yes   
2                No             DSL            Yes  ...               No   
3  No phone service             DSL            Yes  ...              Yes   
4                No     Fiber optic             No  ...               No   

  TechSupport StreamingTV StreamingMovies        Contract Pape

In [25]:
print(df.shape)
print("--------------------------------------------------------------------")
print(df.info())
print("--------------------------------------------------------------------")
print(df.columns)

(7043, 21)
--------------------------------------------------------------------
<class 'pandas.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   str    
 1   gender            7043 non-null   str    
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   str    
 4   Dependents        7043 non-null   str    
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   str    
 7   MultipleLines     7043 non-null   str    
 8   InternetService   7043 non-null   str    
 9   OnlineSecurity    7043 non-null   str    
 10  OnlineBackup      7043 non-null   str    
 11  DeviceProtection  7043 non-null   str    
 12  TechSupport       7043 non-null   str    
 13  StreamingTV       7043 non-null   str    
 14  StreamingMovies   7043 non-null   str    
 15  Contract          70

In [26]:
# Checking Null Values
df['TotalCharges'] = pd.to_numeric(df.TotalCharges, errors='coerce')
print(df.isnull().sum())

# Dropping tenure and fixing total charges.
df.fillna(df["TotalCharges"].mean())
df.drop(labels=df[df['tenure'] == 0].index, axis=0, inplace=True)



customerID           0
gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64


In [27]:
df["SeniorCitizen"]= df["SeniorCitizen"].map({0: "No", 1: "Yes"})
df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0}).astype(int)

In [28]:
df[['tenure', 'MonthlyCharges', 'TotalCharges']].describe()

,tenure,MonthlyCharges,TotalCharges
count,7032.000000,7032.000000,7032.000000
mean,32.421786,64.798208,2283.300441
std,24.545260,30.085974,2266.771362
min,1.000000,18.250000,18.800000
25%,9.000000,35.587500,401.450000
50%,29.000000,70.350000,1397.475000
75%,55.000000,89.862500,3794.737500
max,72.000000,118.750000,8684.800000


In [29]:
# Creating tenure buckets
def tenure_buckets(t):
    if t <= 6:
        return "0-6"
    elif t<=12:
        return "6-12"
    elif t <= 24:
        return "13-24"
    elif t <= 48:
        return "25-48"
    else:
        return "49+"

df["Tenure_bucket"] = df["tenure"].apply(tenure_buckets)
print(df["Tenure_bucket"].value_counts())



Tenure_bucket
49+      2239
25-48    1594
0-6      1470
13-24    1024
6-12      705
Name: count, dtype: int64


In [30]:
# Pushing data to PostgreSQL
from sqlalchemy import create_engine
import psycopg2
import os
from dotenv import load_dotenv

load_dotenv()

conn = psycopg2.connect(
    host=os.getenv("PG_HOST"),
    database=os.getenv("PG_DB"),
    user=os.getenv("PG_USER"),
    password=os.getenv("PG_PASSWORD"),
    port=5432
)

engine = create_engine(
    f"postgresql+psycopg2://{os.getenv('PG_USER')}:{os.getenv('PG_PASSWORD')}"
    f"@{os.getenv('PG_HOST')}:{os.getenv('PG_PORT', '5432')}/{os.getenv('PG_DB')}",
    pool_pre_ping=True,
    pool_recycle=300
)

mapping = {
    "customerID": "customer_id",
    "gender": "gender",
    "SeniorCitizen": "senior_citizen",
    "Partner": "partner",
    "Dependents": "dependents",
    "tenure": "tenure",
    "Tenure_bucket": "tenure_bucket",
    "PhoneService": "phone_service",
    "MultipleLines": "multiple_lines",
    "InternetService": "internet_service",
    "OnlineSecurity": "online_security",
    "OnlineBackup": "online_backup",
    "DeviceProtection": "device_protection",
    "TechSupport": "tech_support",
    "StreamingTV": "streaming_tv",
    "StreamingMovies": "streaming_movies",
    "Contract": "contract",
    "PaperlessBilling": "paperless_billing",
    "PaymentMethod": "payment_method",
    "MonthlyCharges": "monthly_charges",
    "TotalCharges": "total_charges",
    "Churn": "churn"
} 

df = df.rename(columns=mapping)
print(df.columns)

# df.to_sql("churn_dataset", engine, if_exists="replace", index=False)
conn.close()

ids = df['customer_id']


Index(['customer_id', 'gender', 'senior_citizen', 'partner', 'dependents',
       'tenure', 'phone_service', 'multiple_lines', 'internet_service',
       'online_security', 'online_backup', 'device_protection', 'tech_support',
       'streaming_tv', 'streaming_movies', 'contract', 'paperless_billing',
       'payment_method', 'monthly_charges', 'total_charges', 'churn',
       'tenure_bucket'],
      dtype='str')


In [31]:
# One-Hot Encoding
binary_cols = [
    'senior_citizen',
    'partner',
    'dependents',
    'phone_service',
    'paperless_billing',
    'online_security',
    'online_backup',
    'device_protection',
    'tech_support',
    'streaming_tv',
    'streaming_movies'
]
df[binary_cols] = df[binary_cols].replace(
    {'Yes': 1, 'No': 0, 'No internet service': 0}
).astype(int)

df = df.drop(columns=['customer_id'])
categorical_cols = [
    'gender',
    'multiple_lines',
    'internet_service',
    'contract',
    'payment_method',
    'tenure_bucket'
]

df = pd.get_dummies(df, columns=categorical_cols, drop_first=False)

In [32]:
df.columns
bool_cols = df.select_dtypes(include='bool').columns
df[bool_cols] = df[bool_cols].astype(int)
df

,senior_citizen,partner,dependents,tenure,phone_service,online_security,online_backup,device_protection,tech_support,streaming_tv,...,contract_Two year,payment_method_Bank transfer (automatic),payment_method_Credit card (automatic),payment_method_Electronic check,payment_method_Mailed check,tenure_bucket_0-6,tenure_bucket_13-24,tenure_bucket_25-48,tenure_bucket_49+,tenure_bucket_6-12
0,0,1,0,1,0,0,1,0,0,0,...,0,0,0,1,0,1,0,0,0,0
1,0,0,0,34,1,1,0,1,0,0,...,0,0,0,0,1,0,0,1,0,0
2,0,0,0,2,1,1,1,0,0,0,...,0,0,0,0,1,1,0,0,0,0
3,0,0,0,45,0,1,0,1,1,0,...,0,1,0,0,0,0,0,1,0,0
4,0,0,0,2,1,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,0,1,1,24,1,1,0,1,1,1,...,0,0,0,0,1,0,1,0,0,0
7039,0,1,1,72,1,0,1,1,0,1,...,0,0,1,0,0,0,0,0,1,0
7040,0,1,1,11,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
7041,1,1,0,4,1,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0


In [33]:
# Checking Correlation
corr_data = df.corr()['churn'].sort_values(ascending=False)

In [34]:
X = df.drop('churn', axis=1)
y = df['churn'].values

from sklearn.model_selection import train_test_split
import matplotlib as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.30, random_state = 40, stratify=y)

# 1. Logistic Regression
scaler = StandardScaler()
X_scaled_train = scaler.fit_transform(X_train)
X_scaled_test = scaler.transform(X_test)

log_reg = LogisticRegression()
log_reg.fit(X_scaled_train, y_train)
accuracy_lr = log_reg.score(X_scaled_test, y_test)
print("\nAccuracy", accuracy_lr)

y_pred = log_reg.predict(X_scaled_test)
report = classification_report(y_test, y_pred)
print("\nReport: ", report)

cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix: \n", cm)


y_pred_proba = log_reg.predict_proba(X_scaled_test)[:,1]
roc_auc = roc_auc_score(y_test, y_pred_proba)
print("\nROC-AUC Score: ", roc_auc)


Accuracy 0.8151658767772512

Report:                precision    recall  f1-score   support

           0       0.85      0.91      0.88      1549
           1       0.69      0.55      0.61       561

    accuracy                           0.82      2110
   macro avg       0.77      0.73      0.75      2110
weighted avg       0.81      0.82      0.81      2110


Confusion Matrix: 
 [[1409  140]
 [ 250  311]]

ROC-AUC Score:  0.8607093990833027


In [35]:
feature_importance = pd.DataFrame({
    "feature": X_train.columns,
    "coefficient": log_reg.coef_[0]
})

feature_importance["abs_coeff"] = np.abs(feature_importance["coefficient"])
feature_importance = feature_importance.sort_values("abs_coeff", ascending=False)
print(feature_importance)

                                     feature  coefficient  abs_coeff
3                                     tenure    -0.893480   0.893480
21                       internet_service_No    -0.463591   0.463591
20              internet_service_Fiber optic     0.462440   0.462440
24                         contract_Two year    -0.377106   0.377106
12                           monthly_charges    -0.317175   0.317175
22                   contract_Month-to-month     0.313921   0.313921
13                             total_charges     0.238707   0.238707
9                               streaming_tv     0.197090   0.197090
29                         tenure_bucket_0-6     0.193295   0.193295
10                          streaming_movies     0.192454   0.192454
11                         paperless_billing     0.160233   0.160233
27           payment_method_Electronic check     0.130448   0.130448
8                               tech_support    -0.128586   0.128586
30                       tenure_bu

In [36]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=200, random_state=42, class_weight="balanced", max_leaf_nodes=30)
rf.fit(X_train, y_train)
accuracy_rf = rf.score(X_test, y_test)
print("Accuracy: ", accuracy_rf)

y_pred = rf.predict(X_test)
report = classification_report(y_test, y_pred)
print("\nReport: ", report)

cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix: \n", cm)


y_pred_proba = log_reg.predict_proba(X_scaled_test)[:,1]
roc_auc = roc_auc_score(y_test, y_pred_proba)
print("\nROC-AUC Score: ", roc_auc)

Accuracy:  0.7573459715639811

Report:                precision    recall  f1-score   support

           0       0.92      0.73      0.82      1549
           1       0.53      0.82      0.64       561

    accuracy                           0.76      2110
   macro avg       0.72      0.78      0.73      2110
weighted avg       0.81      0.76      0.77      2110


Confusion Matrix: 
 [[1138  411]
 [ 101  460]]

ROC-AUC Score:  0.8607093990833027


In [37]:
feature_importance = pd.DataFrame({
    "feature": X_train.columns,
    "coefficient": rf.feature_importances_
})
feature_importance = feature_importance.sort_values("coefficient", ascending=False)
print(feature_importance)

                                     feature  coefficient
22                   contract_Month-to-month     0.176045
3                                     tenure     0.113582
24                         contract_Two year     0.095919
20              internet_service_Fiber optic     0.094990
13                             total_charges     0.075922
12                           monthly_charges     0.072065
27           payment_method_Electronic check     0.063242
29                         tenure_bucket_0-6     0.049194
21                       internet_service_No     0.043258
32                         tenure_bucket_49+     0.042694
23                         contract_One year     0.026408
19                      internet_service_DSL     0.025180
8                               tech_support     0.014522
11                         paperless_billing     0.013881
5                            online_security     0.013005
10                          streaming_movies     0.009349
2             

In [38]:
# Feature Engineering + Recheck all Models
df['is_new_customer'] = np.where(df['tenure'] <= 6, 1, 0)
df['is_long_tenure'] = np.where(df['tenure'] >= 24, 1, 0)

monthly_charge_median = df['monthly_charges'].median()
df['above_avg_charge'] = np.where(df['monthly_charges'] >= monthly_charge_median, 1, 0)

df['total_charges_tenure_ratio'] = df['total_charges']/df['tenure']
df['price_tenure_interaction'] =  df['monthly_charges'] * df['tenure']

df["is_auto_payment"] = df[[
    "payment_method_Bank transfer (automatic)",
    "payment_method_Credit card (automatic)"
]].max(axis=1)
df

,senior_citizen,partner,dependents,tenure,phone_service,online_security,online_backup,device_protection,tech_support,streaming_tv,...,tenure_bucket_13-24,tenure_bucket_25-48,tenure_bucket_49+,tenure_bucket_6-12,is_new_customer,is_long_tenure,above_avg_charge,total_charges_tenure_ratio,price_tenure_interaction,is_auto_payment
0,0,1,0,1,0,0,1,0,0,0,...,0,0,0,0,1,0,0,29.850000,29.85,0
1,0,0,0,34,1,1,0,1,0,0,...,0,1,0,0,0,1,0,55.573529,1936.30,0
2,0,0,0,2,1,1,1,0,0,0,...,0,0,0,0,1,0,0,54.075000,107.70,0
3,0,0,0,45,0,1,0,1,1,0,...,0,1,0,0,0,1,0,40.905556,1903.50,1
4,0,0,0,2,1,0,0,0,0,0,...,0,0,0,0,1,0,1,75.825000,141.40,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,0,1,1,24,1,1,0,1,1,1,...,1,0,0,0,0,1,1,82.937500,2035.20,0
7039,0,1,1,72,1,0,1,1,0,1,...,0,0,1,0,0,1,1,102.262500,7430.40,1
7040,0,1,1,11,0,1,0,0,0,0,...,0,0,0,1,0,0,0,31.495455,325.60,0
7041,1,1,0,4,1,0,0,0,0,0,...,0,0,0,0,1,0,1,76.650000,297.60,0


In [40]:
df.to_csv('data/final_dataset.csv', index=False)

In [16]:
X = df.drop('churn', axis=1)
y = df['churn'].values

from sklearn.model_selection import train_test_split
import matplotlib as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.30, random_state = 40, stratify=y)

# 1. Logistic Regression
scaler = StandardScaler()
X_scaled_train = scaler.fit_transform(X_train)
X_scaled_test = scaler.transform(X_test)

log_reg = LogisticRegression()
log_reg.fit(X_scaled_train, y_train)
accuracy_lr = log_reg.score(X_scaled_test, y_test)
print("\nAccuracy", accuracy_lr)

y_pred = log_reg.predict(X_scaled_test)
report = classification_report(y_test, y_pred)
print("\nReport: ", report)

cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix: \n", cm)


y_pred_proba = log_reg.predict_proba(X_scaled_test)[:,1]
roc_auc = roc_auc_score(y_test, y_pred_proba)
print("\nROC-AUC Score: ", roc_auc)

feature_importance = pd.DataFrame({
    "feature": X_train.columns,
    "coefficient": log_reg.coef_[0]
})

feature_importance["abs_coeff"] = np.abs(feature_importance["coefficient"])
feature_importance = feature_importance.sort_values("abs_coeff", ascending=False)
print('\n\n',feature_importance)


Accuracy 0.8151658767772512

Report:                precision    recall  f1-score   support

           0       0.85      0.91      0.88      1549
           1       0.69      0.55      0.61       561

    accuracy                           0.82      2110
   macro avg       0.77      0.73      0.75      2110
weighted avg       0.81      0.82      0.81      2110


Confusion Matrix: 
 [[1409  140]
 [ 250  311]]

ROC-AUC Score:  0.8603388535412992


                                      feature  coefficient  abs_coeff
3                                     tenure    -0.860821   0.860821
20              internet_service_Fiber optic     0.465258   0.465258
21                       internet_service_No    -0.460253   0.460253
24                         contract_Two year    -0.380198   0.380198
22                   contract_Month-to-month     0.315904   0.315904
9                               streaming_tv     0.195748   0.195748
10                          streaming_movies     0.193719   0.19

In [17]:
rf = RandomForestClassifier(n_estimators=200, random_state=42, class_weight="balanced", max_leaf_nodes=30)
rf.fit(X_train, y_train)
accuracy_rf = rf.score(X_test, y_test)
print("Accuracy: ", accuracy_rf)

y_pred = rf.predict(X_test)
report = classification_report(y_test, y_pred)
print("\nReport: ", report)

cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix: \n", cm)


y_pred_proba = log_reg.predict_proba(X_scaled_test)[:,1]
roc_auc = roc_auc_score(y_test, y_pred_proba)
print("\nROC-AUC Score: ", roc_auc)

feature_importance = pd.DataFrame({
    "feature": X_train.columns,
    "coefficient": rf.feature_importances_
})
feature_importance = feature_importance.sort_values("coefficient", ascending=False)
print('\n\n',feature_importance)

Accuracy:  0.7592417061611374

Report:                precision    recall  f1-score   support

           0       0.91      0.74      0.82      1549
           1       0.53      0.81      0.64       561

    accuracy                           0.76      2110
   macro avg       0.72      0.77      0.73      2110
weighted avg       0.81      0.76      0.77      2110


Confusion Matrix: 
 [[1150  399]
 [ 109  452]]

ROC-AUC Score:  0.8603388535412992


                                      feature  coefficient
22                   contract_Month-to-month     0.162197
24                         contract_Two year     0.093912
3                                     tenure     0.090868
20              internet_service_Fiber optic     0.084243
12                           monthly_charges     0.056690
37                total_charges_tenure_ratio     0.052279
13                             total_charges     0.048310
32                         tenure_bucket_49+     0.046991
27           payment_met

In [18]:
import xgboost as xgb
xg = xgb.XGBClassifier(random_state=42, scale_pos_weight=4)
xg.fit(X_train, y_train)
y_pred = xg.predict(X_test)
score = xg.score(X_test, y_test)
print("Score: ", score)

report = classification_report(y_test, y_pred)
print("\nReport ", report)

cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix: \n", cm)

Score:  0.7578199052132701

Report                precision    recall  f1-score   support

           0       0.89      0.77      0.82      1549
           1       0.53      0.72      0.61       561

    accuracy                           0.76      2110
   macro avg       0.71      0.75      0.72      2110
weighted avg       0.79      0.76      0.77      2110


Confusion Matrix: 
 [[1193  356]
 [ 155  406]]


In [19]:
from sklearn.ensemble import VotingClassifier
import xgboost as xgb
from sklearn.pipeline import Pipeline
clf1 = Pipeline([
    ("scaler", StandardScaler()),
    ("lr", LogisticRegression(class_weight="balanced", max_iter=1000))
])
clf2 = RandomForestClassifier(n_estimators=200, random_state=42, class_weight="balanced", max_leaf_nodes=30)
clf3 = xgb.XGBClassifier(random_state=42, scale_pos_weight=4)
eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('xgb', clf3)], voting='soft')
eclf.fit(X_train, y_train)
y_pred = eclf.predict(X_test)
score = eclf.score(X_test, y_test)
print("Score: ", score)

report = classification_report(y_test, y_pred)
print("\nReport ", report)

cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix: \n", cm)

Score:  0.7568720379146919

Report                precision    recall  f1-score   support

           0       0.91      0.74      0.82      1549
           1       0.53      0.80      0.64       561

    accuracy                           0.76      2110
   macro avg       0.72      0.77      0.73      2110
weighted avg       0.81      0.76      0.77      2110


Confusion Matrix: 
 [[1147  402]
 [ 111  450]]


In [20]:
# Using the trained model to predict churn probability of all the customers

df['churn_probability'] = rf.predict_proba(X)[:, 1]
df['retention_priority_score'] = df['churn_probability'] * df['monthly_charges']
df

,senior_citizen,partner,dependents,tenure,phone_service,online_security,online_backup,device_protection,tech_support,streaming_tv,...,tenure_bucket_49+,tenure_bucket_6-12,is_new_customer,is_long_tenure,above_avg_charge,total_charges_tenure_ratio,price_tenure_interaction,is_auto_payment,churn_probability,retention_priority_score
0,0,1,0,1,0,0,1,0,0,0,...,0,0,1,0,0,29.850000,29.85,0,0.786418,23.474576
1,0,0,0,34,1,1,0,1,0,0,...,0,0,0,1,0,55.573529,1936.30,0,0.176713,10.063805
2,0,0,0,2,1,1,1,0,0,0,...,0,0,1,0,0,54.075000,107.70,0,0.728128,39.209716
3,0,0,0,45,0,1,0,1,1,0,...,0,0,0,1,0,40.905556,1903.50,1,0.181855,7.692487
4,0,0,0,2,1,0,0,0,0,0,...,0,0,1,0,1,75.825000,141.40,0,0.852655,60.282724
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,0,1,1,24,1,1,0,1,1,1,...,0,0,0,1,1,82.937500,2035.20,0,0.268266,22.748965
7039,0,1,1,72,1,0,1,1,0,1,...,1,0,0,1,1,102.262500,7430.40,1,0.296443,30.592948
7040,0,1,1,11,0,1,0,0,0,0,...,0,1,0,0,0,31.495455,325.60,0,0.573456,16.974287
7041,1,1,0,4,1,0,0,0,0,0,...,0,0,1,0,1,76.650000,297.60,0,0.827877,61.594064


In [21]:
df.sort_values("churn_probability", ascending=False).head()

,senior_citizen,partner,dependents,tenure,phone_service,online_security,online_backup,device_protection,tech_support,streaming_tv,...,tenure_bucket_49+,tenure_bucket_6-12,is_new_customer,is_long_tenure,above_avg_charge,total_charges_tenure_ratio,price_tenure_interaction,is_auto_payment,churn_probability,retention_priority_score
2191,1,0,0,1,1,0,0,0,0,1,...,0,0,1,0,1,91.30,91.30,0,0.882765,80.596467
5783,1,0,0,1,1,0,0,0,0,1,...,0,0,1,0,1,89.30,89.30,0,0.882489,78.806229
1600,1,0,0,1,1,0,0,1,0,1,...,0,0,1,0,1,88.80,88.80,0,0.882478,78.364073
6748,1,0,0,1,1,0,0,1,0,0,...,0,0,1,0,1,85.00,85.00,0,0.882257,74.991886
2397,0,0,0,1,1,0,0,0,0,1,...,0,0,1,0,1,88.35,88.35,0,0.882103,77.933765


In [22]:
# Exporting the final dataset
data = df[['churn_probability', 'retention_priority_score']]
final_df = pd.concat([ids, data], axis=1)
print(final_df)

     customer_id  churn_probability  retention_priority_score
0     7590-VHVEG           0.786418                 23.474576
1     5575-GNVDE           0.176713                 10.063805
2     3668-QPYBK           0.728128                 39.209716
3     7795-CFOCW           0.181855                  7.692487
4     9237-HQITU           0.852655                 60.282724
...          ...                ...                       ...
7038  6840-RESVB           0.268266                 22.748965
7039  2234-XADUH           0.296443                 30.592948
7040  4801-JZAZL           0.573456                 16.974287
7041  8361-LTMKD           0.827877                 61.594064
7042  3186-AJIEK           0.184349                 19.476509

[7032 rows x 3 columns]


In [23]:
from psycopg2.extras import execute_values
def upsert_pred(df, table_name, conn):
    tuples = [tuple(x) for x in df.to_numpy()]
    cols = ",".join(df.columns)

    query = f"""
    INSERT INTO {table_name} ({cols})
        VALUES %s
        ON CONFLICT (customer_id)
        DO UPDATE SET
            churn_probability = EXCLUDED.churn_probability,
            retention_priority_score = EXCLUDED.retention_priority_score;
    """
    with conn.cursor() as cur:
        execute_values(cur, query, tuples)
        conn.commit()

conn = psycopg2.connect(
    host=os.getenv("PG_HOST"),
    database=os.getenv("PG_DB"),
    user=os.getenv("PG_USER"),
    password=os.getenv("PG_PASSWORD"),
    port=5432
)

engine = create_engine(
    f"postgresql+psycopg2://{os.getenv('PG_USER')}:{os.getenv('PG_PASSWORD')}"
    f"@{os.getenv('PG_HOST')}:{os.getenv('PG_PORT', '5432')}/{os.getenv('PG_DB')}",
    pool_pre_ping=True,
    pool_recycle=300
)

upsert_pred(final_df, "churn_dataset", conn)

In [ ]:
# Exporting Models

